In [47]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
%run tinyProp.py    # import all classes and methods

In [11]:
batch_size_train = 20
batch_size_test = 1000
random_seed = 1
torch.manual_seed(random_seed)

# Import dataset
train_set = torchvision.datasets.FashionMNIST("./data", download=True, transform=
                                                transforms.Compose([transforms.ToTensor()]))
test_set = torchvision.datasets.FashionMNIST("./data", download=True, train=False, transform=
                                               transforms.Compose([transforms.ToTensor()]))  
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size_train)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size_test)
def output_label(label):
    output_mapping = {
                 0: "T-shirt/Top",
                 1: "Trouser",
                 2: "Pullover",
                 3: "Dress",
                 4: "Coat", 
                 5: "Sandal", 
                 6: "Shirt",
                 7: "Sneaker",
                 8: "Bag",
                 9: "Ankle Boot"
                 }
    input = (label.item() if type(label) == torch.Tensor else label)
    return output_mapping[input]

In [48]:
# Hyperparameters
n_epochs = 5
batch_size_train = 20
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
tpParams = TinyPropParams(0, 1, 1, 5)   # S_min, S_max, zeta, N_layers


class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # define layers
        self.conv1 = nn.Conv2d(1, 6, 5)                     # normal layer
        self.conv2 = TinyPropConv2d(6, 12, 5, tpParams, 2)  # TinyProp-layer

        self.fc1 = TinyPropLinear(192, 120, tpParams, 3)    # TinyProp-layer
        self.fc2 = nn.Linear(120, 60)                       # normal layer
        self.fc3 = nn.Linear(60, 10)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        x = x.view(-1, 12 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)
        return x

In [ ]:
model = Net()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
loss = F.cross_entropy

# you can evaluate the untrained model to get baseline values
test_loss, test_acc = evaluate(model, loss, test_loader)

for epoch in range(1, n_epochs + 1):
    # train one epoch and retrieve average loss and accuracy. You may safe them for plotting
    train_loss, train_acc = trainOneEpoch(model, optimizer, loss, train_loader, epoch)

    # it is good practise to evaluate on the test set every epoch. Once again, 
    test_loss, test_acc = evaluate(model, loss, test_loader)

In [ ]:
# after the training you can obtain the mean epoch bpr, k directly from the TinyPropLayers (one entry per epoch)

print(model.conv2.epochBpr)
print(model.fc1.epochK)